In [10]:
import json

file = open('translation2019zh_valid1.json' , 'r',encoding='utf-8') 

en_data=[]
ch_data=[]

for line in file.readlines():
    tmp=json.loads(line)
    en_data.append(tmp['english'])
    ch_data.append(tmp['chinese'])

#print(en[5161433])
#print(ch[5161433])
#print(en[:10])
#print(ch[:10])

file.close()

In [11]:
# 分別生成中英文字典
en_vocab = set(''.join(en_data))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(ch_data))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

#print(en2id)

In [12]:
en_num_data = [[en2id[en] for en in line ] for line in en_data]
ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]

print('char:', en_data[1])
print('index:', en_num_data[1])

char: I didn't own a Thesaurus until four years ago and I use a small Webster's dictionary that I'd bought at K-Mart for 89 cents.
index: [110, 214, 73, 81, 73, 100, 89, 54, 214, 184, 57, 100, 214, 72, 214, 96, 156, 136, 20, 72, 5, 123, 5, 20, 214, 5, 100, 54, 81, 48, 214, 34, 184, 5, 123, 214, 80, 136, 72, 123, 20, 214, 72, 199, 184, 214, 72, 100, 73, 214, 110, 214, 5, 20, 136, 214, 72, 214, 20, 127, 72, 48, 48, 214, 213, 136, 31, 20, 54, 136, 123, 89, 20, 214, 73, 81, 191, 54, 81, 184, 100, 72, 123, 80, 214, 54, 156, 72, 54, 214, 110, 89, 73, 214, 31, 184, 5, 199, 156, 54, 214, 72, 54, 214, 17, 158, 0, 72, 123, 54, 214, 34, 184, 123, 214, 84, 178, 214, 191, 136, 100, 54, 20, 126]


In [13]:
import numpy as np

# 獲取輸入輸出端的最大長度
max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float16')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float16')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float16')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 311
max decoder length: 174
index data:
 [110, 214, 73, 81, 73, 100, 89, 54, 214, 184, 57, 100, 214, 72, 214, 96, 156, 136, 20, 72, 5, 123, 5, 20, 214, 5, 100, 54, 81, 48, 214, 34, 184, 5, 123, 214, 80, 136, 72, 123, 20, 214, 72, 199, 184, 214, 72, 100, 73, 214, 110, 214, 5, 20, 136, 214, 72, 214, 20, 127, 72, 48, 48, 214, 213, 136, 31, 20, 54, 136, 123, 89, 20, 214, 73, 81, 191, 54, 81, 184, 100, 72, 123, 80, 214, 54, 156, 72, 54, 214, 110, 89, 73, 214, 31, 184, 5, 199, 156, 54, 214, 72, 54, 214, 17, 158, 0, 72, 123, 54, 214, 34, 184, 123, 214, 84, 178, 214, 191, 136, 100, 54, 20, 126]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.003
BATCH_SIZE = 100
EPOCHS = 200

In [15]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [16]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, 218)    0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, None, 3409)   0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, None, 256),  486400      input_4[0][0]                    
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  3753984     input_5[0][0]                    
                                                                 lstm_7[0][1]               

In [ ]:
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])